In [1]:
import warnings
warnings.filterwarnings('ignore')

# Dasymetric Dot Maps

[Jeff Allen](http://jamaps.github.io)

April, 2023

intro text

image

intro text

## Prerequisites

Prior knowledge of Python, including `pandas` and `geopandas`, as well as Inkscape or similar graphic design software, would be helpful for the following tutorial.

Click [here](https://github.com/schoolofcities/mapping-workshops-2023/raw/main/notebooks/bivariate-maps/bivariate-maps.zip) to download this article as a Jupyter Notebook alongside the datasets required. In the download, there is also a standalone Python script, if you want to run the steps all-at-once or integrate with anything else you have cooking.

You can also run the notebook [Binder](https://mybinder.org/v2/gh/schoolofcities/mapping-workshops-2023/HEAD?urlpath=lab%2Ftree%2Fnotebooks%2Fbivariate-maps%2Fbivariate-maps.ipynb)

If you are running the notebook and/or script locally (generally recommended), you will need to use the following libraries. You'll have to install them (e.g. via `pip` or `conda`) if you do not have them installed already.